In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from numba import njit

from twpasolver.models import TWPA

In [ ]:
@njit
def phase_matching(
    freqs, k_array, pump_freqs, k_array_p, I0=250e-6, Istar=6.3e-3, I=1e-3
):
    delta_k = np.empty(shape=(len(freqs), len(pump_freqs)))
    for (i, j), _ in np.ndenumerate(delta_k):
        k_signal = k_array[i]
        k_pump = k_array_p[j]
        k_idler = k_array[np.searchsorted(freqs, pump_freqs[j] - freqs[i])]
        delta_k[i, j] = abs(
            k_pump
            - k_signal
            - k_idler
            + I0**2 / (8 * (I**2 + Istar**2)) * (k_pump - 2 * k_idler - 2 * k_signal)
        )
    return delta_k


twpa = TWPA.from_file("model_cpw_dartwars_13nm_Lk8_5.json")
N_tot = twpa.N * sum(cell.N for cell in twpa.cells)


all_freqs = np.arange(0, 11, 1e-3)
cell = twpa.get_network(all_freqs)
all_k = cell.s21.s_rad_unwrap.flatten() / N_tot

n_p_min = 7000
n_p_max = -1

freqs_pump = all_freqs[n_p_min:n_p_max]
k_array_p = all_k[n_p_min:n_p_max]

n_s_max = 7000
freqs_s = all_freqs[:n_s_max]
k_array = all_k[:n_s_max]

deltas = phase_matching(
    freqs_s, k_array, freqs_pump, k_array_p, I0=twpa.Ip0, Istar=twpa.Istar
)

In [ ]:
thin = 10
plt.pcolor(
    freqs_s[::thin],
    freqs_pump[::thin],
    np.log(deltas[::thin, ::thin].transpose()),
)
c = plt.colorbar()
c.ax.set_xlabel(r"log|$\Delta_\beta$|")
plt.ylabel("pump frequency [GHz]")
plt.xlabel("signal frequency [GHz]")
# plt.savefig("phase_matching.png", dpi=150)